In [21]:
# Import libraries

import json
import pandas as pd
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
import minsearch

In [2]:
# Read JSON document with generated ids

with open('documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [3]:
# Load the ground truth CSV file into a DataFrame
df_ground_truth = pd.read_csv('ground-truth-data.csv')

# Convert DataFrame to a list of dictionaries
ground_truth = df_ground_truth.to_dict(orient='records')

In [5]:
ground_truth[1]

{'question': 'How can I get the course schedule?',
 'course': 'data-engineering-zoomcamp',
 'document': 'c02e79ef'}

**Elastic Search**

In [6]:
# Initialize elastic search

es_client = Elasticsearch('http://localhost:9200') 

In [7]:
# Create an index with an id evaluation property

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [8]:
# Index the documents

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|█████████████| 948/948 [03:57<00:00,  4.00it/s]


In [9]:
def elastic_search(query, course):
    # Define the search query with weighted fields and course filter
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text^3"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    # Extract and return the result documents
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [10]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = elastic_search(query=q['question'], course=q['course'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

100%|███████████| 4627/4627 [04:49<00:00, 15.99it/s]


*- Hit-Rate (recall)*

*- Mean Reciprocal Rank (mrr)*

In [11]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [12]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [13]:
hit_rate(relevance_total), mrr(relevance_total)

(0.9135508969094446, 0.836074490310496)

**Using Minsearch**

In [14]:
index = minsearch.Index(
    text_fields=["question", "text"],
    keyword_fields=["course", "id"]
)

index.fit(documents)

In [15]:
def minsearch_search(query, course):
    boost = {'question': 3.0, 'text' : 3.0}

    results = index.search(
        query=query,
        filter_dict={'course': course},
        boost_dict=boost,
        num_results=5
    )

    return results

In [16]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = minsearch_search(query=q['question'], course=q['course'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

100%|██████████| 4627/4627 [00:34<00:00, 134.56it/s]


In [17]:
hit_rate(relevance_total), mrr(relevance_total)

(0.8809163604927599, 0.7660543188531092)

In [18]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [19]:
evaluate(ground_truth, lambda q: elastic_search(q['question'], q['course']))

100%|███████████| 4627/4627 [04:33<00:00, 16.94it/s]


{'hit_rate': 0.9135508969094446, 'mrr': 0.836074490310496}

In [20]:
evaluate(ground_truth, lambda q: minsearch_search(q['question'], q['course']))

100%|██████████| 4627/4627 [00:35<00:00, 130.04it/s]


{'hit_rate': 0.8809163604927599, 'mrr': 0.7660543188531092}